<a href="https://colab.research.google.com/github/surenkutty/Sign-Language-detection-main/blob/main/asl_sign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # Upload kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"surendkumar","key":"5d33baddb7b0d6cf8ce8d21d9a93294f"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d grassknoted/asl-alphabet


Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
 99% 1.01G/1.03G [00:06<00:00, 211MB/s]
100% 1.03G/1.03G [00:06<00:00, 158MB/s]


In [4]:
import zipfile

with zipfile.ZipFile("asl-alphabet.zip", "r") as zip_ref:
    zip_ref.extractall("/content/asl_dataset")

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [6]:
pip install keras


In [8]:
pip install --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 w

In [1]:
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive


In [2]:
drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/asl_model.h5"  # Save model to Google Drive
labels_path = "/content/drive/MyDrive/asl_labels.json"  # Save labels


Mounted at /content/drive


In [3]:
import shutil

train_path = "/content/asl_dataset/asl_alphabet_train/asl_alphabet_train"
fixed_train_path = "/content/asl_dataset/asl_alphabet_train_fixed"

if not os.path.exists(fixed_train_path):
    os.makedirs(fixed_train_path)

for label in os.listdir(train_path):
    src = os.path.join(train_path, label)
    dst = os.path.join(fixed_train_path, label)
    shutil.move(src, dst)

print("Fixed dataset structure!")


Fixed dataset structure!


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to fixed dataset
train_path = "/content/asl_dataset/asl_alphabet_train_fixed"

# Image preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize pixel values
    rotation_range=10,     # Data augmentation
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    validation_split=0.2   # 20% for validation
)

# Load training & validation data
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),   # Resize images
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Class Labels: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_labels), activation='softmax')  # Output layer
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 736s 337ms/step - accuracy: 0.3870 - loss: 2.0618 - val_accuracy: 0.6609 - val_loss: 1.1107
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 724s 329ms/step - accuracy: 0.8491 - loss: 0.4435 - val_accuracy: 0.7785 - val_loss: 0.7776
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 713s 328ms/step - accuracy: 0.9165 - loss: 0.2460 - val_accuracy: 0.7994 - val_loss: 0.7089
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 716s 329ms/step - accuracy: 0.9391 - loss: 0.1805 - val_accuracy: 0.8144 - val_loss: 0.7011
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 706s 324ms/step - accuracy: 0.9537 - loss: 0.1387 - val_accuracy: 0.8069 - val_loss: 0.7495
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 698s 321ms/step - accuracy: 0.9638 - loss: 0.1142 - val_accuracy: 0.8461 - val_loss: 0.6481
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 708s 326ms/step - accuracy: 0.9675 - loss: 0.1022 - val_accuracy: 0.8238 - val_loss: 0.8417
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 700s 322ms/step - ac

In [6]:
# Save model
model.save('/content/drive/MyDrive/asl_model.h5')

# Save class labels
import json

with open('/content/drive/MyDrive/asl_labels.json', 'w') as f:
    json.dump(class_labels, f)

print("Model and labels saved successfully!")


Model and labels saved successfully!


In [8]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import json

# Load model
model = load_model('/content/drive/MyDrive/asl_model.h5')

# Load labels
with open('/content/drive/MyDrive/asl_labels.json', 'r') as f:
    class_labels = json.load(f)

# Function to predict image
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    print(f"Predicted Class: {predicted_class}")

# Test on an image
test_image_path = "/content/asl_dataset/asl_alphabet_test/asl_alphabet_test/A_test.jpg"
predict_image(test_image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
Predicted Class: A


In [9]:
!pip install tensorflow opencv-python pyttsx3